In [1]:
import tensorflow as tf
import keras
import numpy as np
import os

2024-05-17 15:07:36.793528: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-17 15:07:36.798533: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 15:07:36.895792: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-17 15:07:36.897460: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-17 15:07:38.628500: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
text=open('data/gabo.txt').read().lower()

In [3]:
len(text)

814632

In [4]:
seq_length = 60
step = 1

Building the dataset

In [5]:
def generate_sequences(step):
    sequences = []
    next_char = []

    for i in range(0, len(text) - seq_length, step):
        sequences.append(text[i: i + seq_length])
        next_char.append(text[i + seq_length])

    num_seq = len(sequences)
    print('Number of sequences:', num_seq, '\n')

    return sequences, next_char, num_seq

In [6]:
seq_length = 60
step = 1
sequences, next_char, num_seq = generate_sequences(step)

Number of sequences: 814572 



In [7]:
sequences[0], next_char[0]

('muchos años después, frente al pelotón de fusilamiento, el c', 'o')

In [8]:
chars=sorted(list(set(text)))
len(chars)

62

In [9]:
char_indices = dict((char, chars.index(char)) for char in chars)
char_indices

{'\n': 0,
 ' ': 1,
 '!': 2,
 "'": 3,
 '(': 4,
 ')': 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 ';': 20,
 '<': 21,
 '?': 22,
 'a': 23,
 'b': 24,
 'c': 25,
 'd': 26,
 'e': 27,
 'f': 28,
 'g': 29,
 'h': 30,
 'i': 31,
 'j': 32,
 'k': 33,
 'l': 34,
 'm': 35,
 'n': 36,
 'o': 37,
 'p': 38,
 'q': 39,
 'r': 40,
 's': 41,
 't': 42,
 'u': 43,
 'v': 44,
 'w': 45,
 'x': 46,
 'y': 47,
 'z': 48,
 '~': 49,
 '¡': 50,
 '«': 51,
 '»': 52,
 '¿': 53,
 'á': 54,
 'é': 55,
 'í': 56,
 'ñ': 57,
 'ó': 58,
 'ú': 59,
 'ü': 60,
 '⁄': 61}

In [10]:
x=np.zeros( (len(sequences), seq_length, len(chars)), dtype=bool )
y=np.zeros( (len(sequences),len(chars)), dtype=bool )

In [11]:
x[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [12]:
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t, char_indices[char]] = 1
    y[i, char_indices[next_char[i]]] = 1

In [14]:
x[0][0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

Estas matrices preparadas serán utilizadas como entrada y salida esperada para entrenar el modelo de aprendizaje automático. Específicamente:

    La matriz x representa las secuencias de entrada, donde cada carácter se codifica como un vector binario de longitud igual al número de caracteres únicos. Esto se conoce como codificación one-hot.
    La matriz y representa las salidas esperadas, donde cada fila corresponde a una secuencia y cada columna representa un carácter. El valor de 1 indica el siguiente carácter esperado después de cada secuencia.

Building the Model

In [15]:
from keras.layers import Dense, Bidirectional, GRU, LSTM, Embedding, Dropout
from keras.models import Model, Sequential
from keras.optimizers import RMSprop

In [19]:
model = Sequential()

model.add(GRU(256, return_sequences=True, input_shape=(seq_length, len(chars))))
model.add(Bidirectional(GRU(128)))
model.add(Dropout(0.2))
model.add(Dense(len(chars), activation='softmax'))

2024-05-16 20:13:52.519084: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-16 20:13:52.522202: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-16 20:13:52.524376: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 256)               245760    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 62)                15934     
                                                                 
Total params: 261,694
Trainable params: 261,694
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(x, y, epochs=1, batch_size=128, shuffle=True)

2024-05-17 15:09:48.402847: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-17 15:09:48.406514: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-17 15:09:48.410482: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

6364/6364 [==============================] - 998s 156ms/step - loss: 1.5048 - accuracy: 0.5396


In [22]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

In [23]:
import random
import sys

In [31]:
start_index=random.randint(0, len(text)-seq_length-1)
generated_text=text[start_index: start_index+seq_length]

input_text = ' el coronel se fue de la casa'
while len(input_text) < seq_length:
    input_text += " "

for temperature in [0.2, 0.5, 1, 1.5]:
    print('\n--- temperature: ', temperature)
    sys.stdout.write(input_text)

    for i in range(100):
        sampled=np.zeros((1,seq_length,len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1

        preds= model.predict(sampled, verbose=0)[0]
        next_index=sample(preds, temperature)
        next_char=chars[next_index]

        generated_text+=next_char
        generated_text=generated_text[1:]

        sys.stdout.write(next_char)


--- temperature:  0.2
 el coronel se fue de la casa                               pués de la casa, y se había de despertido de la consiguió por la casa de la sienta de la casa, y se 
--- temperature:  0.5
 el coronel se fue de la casa                               aspertó que la pormanió a prometidos de aureliano segundo en la consitario en la cara sinuró que la 
--- temperature:  1
 el coronel se fue de la casa                               salgróu el reister, con reacido que su engunró. atro ajas-
el
dispándino, en la ignora de prec-patr,
--- temperature:  1.5
 el coronel se fue de la casa                               
-lo gragoble al prebeco con cuaco en lgegando. vubiría
gompaz4iobalidar se tragnió
-acuz fís que qu

In [34]:
#model.save('models/write_model.h5')

In [13]:
from keras.models import load_model
model= load_model('models/write_model.h5')

2024-05-17 15:08:12.895440: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-05-17 15:08:13.719132: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-17 15:08:13.722257: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-17 15:08:13.725027: I tensorflow/core/common_runtime/executor.cc